In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime

In [2]:
df = pd.read_json('data/data.json')

In [3]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,14327,14328,14329,14330,14331,14332,14333,14334,14335,14336
acct_type,fraudster_event,premium,premium,premium,premium,premium,premium,premium,premium,premium,...,premium,premium,premium,premium,premium,fraudster_event,premium,premium,tos_warn,fraudster_event
approx_payout_date,1266062400,1296720000,1296172800,1388966400,1297900800,1300496400,1297909800,1296709200,1298185200,1297314000,...,1369783800,1369159200,1372010400,1363323600,1370736000,1361689200,1365566400,1368781200,1361361600,1363316400
body_length,3852,3499,2601,12347,2417,117,28,974,4388,974,...,6007,8149,11930,240,4560,667,5812,557,10873,0
channels,5,0,8,6,11,6,5,12,4,12,...,8,5,8,11,5,0,5,13,8,0
country,US,US,US,IE,US,US,US,US,US,US,...,US,GB,GB,US,US,US,US,,US,GB
currency,USD,USD,USD,EUR,USD,USD,USD,USD,USD,USD,...,USD,GBP,GBP,USD,USD,USD,USD,USD,USD,GBP
delivery_method,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
description,"<p><a href=""http://s432.photobucket.com/albums...","<p>Join us for a quick, one-night, community-b...","<h3><span class=""subcategory""><strong>Teacher ...","<p style=""margin-bottom: 1.3em; padding-bottom...",<p>Writers and filmmakers need to understand t...,<p>Come join the Bluegrass Stallions as they t...,<p> </p>\r\n<p> </p>\r\n<p> </p>,<p><em>WHEN JEKYLL MET HYDE</em></p>\r\n<p>The...,"<p style=""text-align: center;""><font face=""boo...",<p><em>WHEN JEKYLL MET HYDE</em></p>\r\n<p>The...,...,"<h2 style=""text-align: center;""><strong><span ...","<p><strong style=""font-size: medium;"">What you...","<p><strong style=""font-size: medium;"">What you...","<p><img alt=""An Evening of Sheer Elegance, Fa...","<p class=""MsoNormal"" style=""margin: 0in 0in 0....",<p>Celebrates this wonderful holiday with the ...,"<table style=""width: 600px;"" border=""0"" cellsp...",<p>What a way to start off your Mother's Day l...,"<p style=""font-family: 'Helvetica Neue', Helve...",
email_domain,gmail.com,ruf.org,pvsd.k12.ca.us,irishtabletennis.com,artsandbusinesscouncil.org,bluegrassstallions.com,naboe.org,themagnetictheatre.org,credosf.com,themagnetictheatre.org,...,stcloudstate.edu,chatterboxsocialmedia.co.uk,chatterboxsocialmedia.co.uk,gmail.com,revolutiongroup.com,yahoo.com,me.com,yahoo.com,velvetlist.com,gmail.com
event_created,1262739706,1293832670,1291090956,1360681570,1291994666,1294421810,1294425018,1294427837,1294428122,1294428286,...,1359746862,1359809733,1359809804,1359824184,1364417805,1360297993,1360367042,1360600330,1360376285,1360686846


### Previous Payouts Explore

In [4]:
len(df.loc[1, 'previous_payouts'])

49

In [5]:
df['previous_payouts'].apply(lambda x: len(x))

0          0
1         49
2         36
3         21
4         49
        ... 
14332      0
14333      5
14334      1
14335    128
14336      0
Name: previous_payouts, Length: 14337, dtype: int64

In [6]:
df['n_payouts'] = df['previous_payouts'].apply(lambda x: len(x))

In [7]:
df.loc[1, 'previous_payouts'][0]['uid']

5419068

In [8]:
np.unique([payee['uid'] for payee in df.loc[1, 'previous_payouts']])

array([5419068])

In [9]:
datetime.datetime.fromtimestamp(df.loc[2, 'approx_payout_date'])

datetime.datetime(2011, 1, 27, 16, 0)

In [10]:
# Make fraud column 
df['fraud'] = df.acct_type.apply(lambda x: 1 if 'fraud' in x else 0)

# Get columns 
columns = list(df.columns)

## Model

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [15]:
df

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,n_payouts,fraud
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,...,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL,0,1
1,premium,1296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,...,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC,49,0
2,premium,1296172800,2601,8,US,USD,1.0,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1291090956,...,1272559388,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA,36,0
3,premium,1388966400,12347,6,IE,EUR,1.0,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1360681570,...,1283870102,3,,None,NaN,NaN,None,None,21,0
4,premium,1297900800,2417,11,US,USD,0.0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1291994666,...,1288984065,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA,49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14332,fraudster_event,1361689200,667,0,US,USD,0.0,<p>Celebrates this wonderful holiday with the ...,yahoo.com,1360297993,...,1360297993,1,6100 North Charles Street,US,39.373780,-76.629921,Elkridge Club,MD,0,1
14333,premium,1365566400,5812,5,US,USD,1.0,"<table style=""width: 600px;"" border=""0"" cellsp...",me.com,1360367042,...,1241625346,4,1775 E Mission Bay Dr,US,32.778906,-117.209791,Hilton San Diego Resort and Spa (Hilton Missio...,CA,5,0
14334,premium,1368781200,557,13,,USD,0.0,<p>What a way to start off your Mother's Day l...,yahoo.com,1360600330,...,1353386971,4,5991 Bullard Road,US,30.041819,-89.957130,Fleur De Lis Ballroom NUMBER 2,LA,1,0
14335,tos_warn,1361361600,10873,8,US,USD,0.0,"<p style=""font-family: 'Helvetica Neue', Helve...",velvetlist.com,1360376285,...,1290204991,3,2297 Cedar Ave,US,40.862283,-73.911363,Salsa con fuego,NY,128,0


In [16]:
mask = df.previous_payouts.apply(lambda x: 0 if len(x) <= 0 else 1)
df['mask_previous_payout'] = mask
X = df[['mask_previous_payout', 'user_age']]
y = df['fraud']

In [17]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)
model = LogisticRegression()
model.fit(Xtrain, ytrain)

LogisticRegression()

In [18]:
model.predict(Xtest)

array([0, 0, 0, ..., 0, 0, 0])

In [58]:
import pickle

In [62]:
model_path = "model.pickle"

In [64]:
pickle.dump(model, open(model_path, 'wb'))

In [65]:
new_model = pickle.load(open(model_path, 'rb'))

In [66]:
new_model.predict_proba(Xtest)

array([[0.96513266, 0.03486734],
       [0.96502979, 0.03497021],
       [0.96513266, 0.03486734],
       ...,
       [0.96502979, 0.03497021],
       [0.98256968, 0.01743032],
       [0.96559197, 0.03440803]])

In [135]:
model2 = pickle.load(open('models/model.pickle', 'rb'))

/Users/vinsonmccready/miniforge3/envs/tf/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [138]:
model2.predict_proba(Xtest)

array([[0.03549236, 0.96450764],
       [0.03561665, 0.96438335],
       [0.03549236, 0.96450764],
       ...,
       [0.03561665, 0.96438335],
       [0.01556803, 0.98443197],
       [0.03493821, 0.96506179]])

In [31]:
import requests

In [42]:
from src.api_client import EventAPIClient

In [94]:
data_path = 'data/api_data.json'

In [95]:
pd.DataFrame([pd.Series(data['data'][0])]).to_json(data_path)

In [96]:
df2 = pd.read_json(data_path)

In [103]:
def get_X(df):
    mask = df.previous_payouts.apply(lambda x: 0 if len(x) <= 0 else 1)
    df['mask_previous_payout'] = mask
    X = df[['mask_previous_payout', 'user_age']]
    return X

In [104]:
new_model.predict_proba(X2)

array([[0.98380536, 0.01619464]])

In [151]:
data['data'][0]

{'body_length': 2267,
 'channels': 8,
 'country': 'CA',
 'currency': 'CAD',
 'delivery_method': 1.0,
 'description': '<p><span style="font-size: large;">This registration encompasses:</span></p>\r\n<ul>\r\n<li><span style="font-size: large;">Attendance or access to the recordings of the three free Hosted Conversations (April, May and June 2013)</span></li>\r\n<li><span style="font-size: large;">3 tele-classes (Sept. 19th, Oct. 3rd and Oct. 17th 2013)</span></li>\r\n<li><span style="font-size: large;">Tuition, room and board* at the 6-day retreat Nov. 11th through 17th, 2013.</span></li>\r\n</ul>\r\n<p><font size="4">* Double occupancy</font></p>\r\n<p><span style="font-size: large; font-family: arial, helvetica, sans-serif;"><br /></span></p>\r\n<p><span style="font-size: large; font-family: arial, helvetica, sans-serif;">For information about the full Program:<br /></span></p>\r\n<div style="text-align: -webkit-auto;"><span style="font-family: arial, helvetica, sans-serif; font-size: 

In [19]:
import pymongo
import time
import requests

In [20]:
db_client = pymongo.MongoClient('localhost', 27017)
db = db_client['fraud']
cases = db.cases

In [150]:
cases.insert_one(data['data'][0])

DuplicateKeyError: E11000 duplicate key error collection: fraud.cases index: _id_ dup key: { _id: ObjectId('6078caa268e6c0707967bc68') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6078caa268e6c0707967bc68')}, 'errmsg': "E11000 duplicate key error collection: fraud.cases index: _id_ dup key: { _id: ObjectId('6078caa268e6c0707967bc68') }"}

In [29]:
first_sequence_number=0
next_sequence_number = first_sequence_number

In [32]:

api_url='https://hxobin8em5.execute-api.us-west-2.amazonaws.com/api/'
api_key='vYm9mTUuspeyAWH1v-acfoTlck-tCxwTw9YfCynC'


def get_data(next_sequence_number):
    """Fetch data from the API."""
    payload = {'api_key': api_key,
               'sequence_number': next_sequence_number}
    response = requests.post(api_url, json=payload)
    data = response.json()
    next_sequence_number = data['_next_sequence_number']
    return data['data']

interval=30
while True:
    print("Requesting data...")
    data = get_data(next_sequence_number)
    if data:
        print("Saving...")
        for row in data:
            cases.insert_one(row)
    else:
        print("No new data received.")
    print(f"Waiting {interval} seconds...")
    time.sleep(interval)

Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Waiting 30 seconds...
Requesting data...
Saving...
Wa

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [39]:
api_data = pd.DataFrame(list(cases.find())).drop_duplicates('object_id').reset_index()

In [42]:
api_data = api_data.drop(['_id', 'index'], axis=1)

In [43]:
api_data.to_json('data/api_data.json')

In [44]:
api_df =  pd.read_json('data/api_data.json') 

In [45]:
api_df

,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,...,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,sequence_number
0,2267,8,CA,CAD,1,"<p><span style=""font-size: large;"">This regist...",spiralis.ca,1359490099,1366516800,1359490251,...,517,1314799174,3,Ecology Retreat Center,CA,43.919979,-80.094311,3 Tele-classes + a 6-day residential retreat,Ontario,2491
1,3109,6,US,USD,0,"<p style=""text-align: justify;""><span style=""c...",yahoo.com,1365689629,1366522200,1365699150,...,371,1333660943,4,400 4th Street,US,32.451633,-84.988110,Columbus Civic Center,GA,2551
2,3107,12,US,USD,0,"<p><span style=""font-size: x-large;""><span sty...",yahoo.com,1365890353,1369976400,1365890402,...,792,1297448548,4,3540 Toulouse Street,US,29.974981,-90.092041,,,2552
3,15036,8,,CAD,0,"<p style=""font-family: 'Helvetica Neue', Helve...",hotmail.com,1365893145,1369369800,1365894159,...,150,1352952859,3,4144 Wilkinson Road,CA,48.479240,-123.410328,Cherries Bistro,British Columbia,2553
4,2010,8,US,USD,0,"<p style=""color: #000000;""><font face=""georgia...",generalassemb.ly,1308696562,1309919400,1308864868,...,105,1299651441,3,"902 Broadway, 4th Floor",US,40.739300,-73.989418,,,2711
5,3702,8,US,USD,0,"<p><img style=""float: right;"" alt=""Mobile Mar...",mmaglobal.com,1308709465,1312414200,1309361486,...,425,1271955013,3,300 West Adams,US,41.879514,-87.635720,"Vibes Media, Inc.",IL,2712
6,1403,6,US,USD,1,"<p><font size=""3"">Vogue Magazine, Style 360, a...",gmail.com,1309145477,1311404400,1309147698,...,51,1304726699,3,415 Gravier St.,US,29.950367,-90.066927,Exclusive Private Loft,LA,2713
7,3215,8,GB,GBP,0,"<p style=""margin: 0px; padding: 0px 0px 8px; f...",breakwaterfp.co.uk,1346415992,1348781400,1346427157,...,1088,1252421102,4,,GB,50.363113,-4.136344,The Royal Corinthian Yacht Club,,2714
8,608,5,US,USD,0,"<div></div>\r\n<div style=""text-align: center;...",nycap.rr.com,1309450850,1312693200,1309451593,...,142,1297222849,1,17 New Scotland Ave,US,42.654263,-73.773160,Valentine's,NY,2715
9,3172,11,CA,CAD,0,"<div class=""mbl notesBlogText clearfix"">\r\n<d...",kortaggio.com,1310311965,1319934600,1312302797,...,0,1310311964,3,100 McCaul Street,CA,43.653424,-79.391069,"OCAD University - Central Hall, room 230",Ontario,2716


In [120]:
def not_premium(x):
    return 1 if x.lower() != 'premium' else 0

def no_previous_payout(x):
    return 1 if len(x) <= 0 else 0

def payout_name_flag(x):
    for d in x:
        if len(d['name']) > 3:
            return 1
    return 0

def payout_toself(payee, payouts):
    for pay in payouts:
        if payee.lower() in pay['name'].lower():
            return 1
    return 0

# ---------------------------------------------------------------------------
# End Pipeline Funtions 
# ---------------------------------------------------------------------------
def data_pipeline(df):
    temp = dict()
    temp = {
#         'target': df['fraud'].copy(), 
#             'premium': df.acct_type.apply(lambda x: not_premium(x)),
            'previous_payout': df.previous_payouts.apply(lambda x: no_previous_payout(x)),
            'no_payout_name': df.previous_payouts.apply(lambda x: payout_name_flag(x)),
            'payout_toself': df.apply(lambda x: payout_toself(x.payee_name, x.previous_payouts), axis = 1)
        }
        
    return pd.DataFrame(temp)

In [121]:
data_pipeline(pd.DataFrame([pd.Series(data['data'][0])]))

,previous_payout,no_payout_name,payout_toself
0,0,0,1


In [145]:
for c in pd.DataFrame([pd.Series(data['data'][0])]).columns:
    print(c)

body_length
channels
country
currency
delivery_method
description
email_domain
event_created
event_end
event_published
event_start
fb_published
has_analytics
has_header
has_logo
listed
name
name_length
object_id
org_desc
org_facebook
org_name
org_twitter
payee_name
payout_type
previous_payouts
sale_duration
show_map
ticket_types
user_age
user_created
user_type
venue_address
venue_country
venue_latitude
venue_longitude
venue_name
venue_state
sequence_number


In [119]:
pd.DataFrame([pd.Series(data['data'][0])])['description'].values[0]

'<p><span style="font-size: large;">This registration encompasses:</span></p>\r\n<ul>\r\n<li><span style="font-size: large;">Attendance or access to the recordings of the three free Hosted Conversations (April, May and June 2013)</span></li>\r\n<li><span style="font-size: large;">3 tele-classes (Sept. 19th, Oct. 3rd and Oct. 17th 2013)</span></li>\r\n<li><span style="font-size: large;">Tuition, room and board* at the 6-day retreat Nov. 11th through 17th, 2013.</span></li>\r\n</ul>\r\n<p><font size="4">* Double occupancy</font></p>\r\n<p><span style="font-size: large; font-family: arial, helvetica, sans-serif;"><br /></span></p>\r\n<p><span style="font-size: large; font-family: arial, helvetica, sans-serif;">For information about the full Program:<br /></span></p>\r\n<div style="text-align: -webkit-auto;"><span style="font-family: arial, helvetica, sans-serif; font-size: large;"><font color="#666666" face="sans-serif"><span style="line-height: 16px;">trainingnvctrainers.com</span></font